In [1]:
from sec_certs.dataset import CCDataset
from pathlib import Path
import shutil
import subprocess
from sec_certs.utils import helpers
from urllib.parse import unquote_plus, urlparse

In [ ]:
def hash_file(path):
    return helpers.get_sha256_filepath(path)
def extract_filename(link):
    return unquote_plus(str(urlparse(link).path).split("/")[-1])

In [2]:
# A directory with a dump of the documents that will enrich the dataset.
doc_dump_dir = Path("cc_certs_09_10_2022")
print(doc_dump_dir.exists())

# An output directory for the dataset.
dataset_dir = Path("cc_09_10_2022")
print(dataset_dir.exists())

True
True


In [3]:
# Print the current commit hash
subprocess.run(["git", "rev-parse", "--short", "HEAD"])

95d1ec0


CompletedProcess(args=['git', 'rev-parse', '--short', 'HEAD'], returncode=0)

In [4]:
# Print tool versions
subprocess.run(["pdftotext", "-v"])
subprocess.run(["tesseract", "-v"]);

tesseract 5.2.0
 leptonica-1.82.0
  libgif 5.2.1 : libjpeg 8d (libjpeg-turbo 2.1.3) : libpng 1.6.38 : libtiff 4.4.0 : zlib 1.2.12 : libwebp 1.2.4 : libopenjp2 2.5.0
 Found AVX2
 Found AVX
 Found FMA
 Found SSE4.1
 Found OpenMP 201511
 Found libarchive 3.6.1 zlib/1.2.12 liblzma/5.2.5 bz2lib/1.0.8 liblz4/1.9.3 libzstd/1.5.2
 Found libcurl/7.85.0 OpenSSL/1.1.1q zlib/1.2.12 brotli/1.0.9 zstd/1.5.2 libidn2/2.3.3 libpsl/0.21.1 (+libidn2/2.3.0) libssh2/1.10.0 nghttp2/1.50.0


pdftotext version 22.09.0
Copyright 2005-2022 The Poppler Developers - http://poppler.freedesktop.org
Copyright 1996-2011, 2022 Glyph & Cog, LLC


In [5]:
# Create the dataset
dset = CCDataset(root_dir=dataset_dir, name="cc", description="Final run on 09.10.2022")

In [6]:
# Get the metadata
dset.get_certs_from_web()

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:04<00:00,  2.00s/it]
The CSV cc_09_10_2022/web/cc_products_active.csv contains 8 duplicates by the primary key.
The CSV cc_09_10_2022/web/cc_products_archived.csv contains 10 duplicates by the primary key.
When merging certificates with dgst b26ce64e0c677e3d, the following mismatch occured: Attribute=security_level, self[security_level]={'ALC_DVS.1', 'EAL2+'}, other[security_level]={'EAL2'}
When merging certificates with dgst 7f043162f88a1c3a, the following mismatch occured: Attribute=not_valid_after, self[not_valid_after]=2024-09-19, other[not_valid_after]=2024-09-18
When merging certificates with dgst 109eb2158ca6a2f9, the following mismatch occured: Attribute=security_level, self[security_level]={'ADV_IMP.2', 'AVA_VLA.4', 'AVA_MSU.3', 'EAL4+', 'ATE_DPT.2'}, other[security_level]={'ADV_DVS.2', 'AVA_VLA.4', 'AVA_MSU.3', 'EAL5+'}
When merging certifi

In [7]:
# Process the profiles?
dset.process_protection_profiles()

Duplicate entry in PP dataset: ('Protection Profile for Enterprise Security Management Policy Management', 'https://www.commoncriteriaportal.org/files/ppfiles/PP_ESM_PM_V1.4.pdf')
Duplicate entry in PP dataset: ('Stateful Traffic Filter Firewall Extended Package for Network Device Protection Profile', 'https://www.commoncriteriaportal.org/files/ppfiles/PP_ND_TFFWEP_V1.0.pdf')
Duplicate entry in PP dataset: ('Stateful Traffic Filter Firewall Extended Package for Network Device Protection Profile', 'https://www.commoncriteriaportal.org/files/ppfiles/PP_ND_TFFWEP_V1.0.pdf')
Duplicate entry in PP dataset: ('Public Key-Enabled Application Family of Protection Profiles', 'https://www.commoncriteriaportal.org/files/ppfiles/PP_VID3004-PP.pdf')
Duplicate entry in PP dataset: ('Protection Profile for Wireless Local Area Network (WLAN) Access Systems', 'https://www.commoncriteriaportal.org/files/ppfiles/PP_WLAN_AS_V1.0.pdf')
Duplicate entry in PP dataset: ('PC Client Specific Trusted Platform Mod

In [8]:
# Enrich the dataset with the pre-downloaded PDFs (collected over multiple runs to fill in the timeouts and disappearances).
dset.reports_pdf_dir.mkdir(parents=True, exist_ok=True)
dset.targets_pdf_dir.mkdir(parents=True, exist_ok=True)
for cert in dset:
    report_pdf = doc_dump_dir / "report" / "pdf" / f"{cert.dgst}.pdf"
    if report_pdf.exists():
        shutil.copy(report_pdf, cert.state.report_pdf_path)
        cert.state.report_download_ok = True
        cert.state.report_pdf_hash = hash_file(cert.state.report_pdf_path)
    cert.pdf_data.report_filename = extract_filename(cert.report_link)

    target_pdf = doc_dump_dir / "target" / "pdf" / f"{cert.dgst}.pdf"
    if target_pdf.exists():
        shutil.copy(target_pdf, cert.state.st_pdf_path)
        cert.state.st_download_ok = True
        cert.state.st_pdf_hash = hash_file(cert.state.st_pdf_path)
    cert.pdf_data.st_filename = extract_filename(cert.st_link)

In [9]:
# Try to download last missing (or ones not present in the pre-downloaded)
dset.download_all_pdfs(fresh=False)

Cert dgst: 02482228eb547c15 failed to download report from https://www.commoncriteriaportal.org/files/epfiles/st_vid4016a-vr.pdf, code: nok
Cert dgst: d79ecebfe21f2144 failed to download report from https://www.commoncriteriaportal.org/files/epfiles/st_vid10024a-vr.pdf, code: nok
Cert dgst: a90d78924e91dc6c failed to download report from https://www.commoncriteriaportal.org/files/epfiles/ANSSI-CC_2010-03en1.pdf, code: nok
Cert dgst: 8ac22970bd8042d2 failed to download report from https://www.commoncriteriaportal.org/files/epfiles/st_vid4016b-vr.pdf, code: nok
Cert dgst: e27eb9efbd67c4f7 failed to download report from https://www.commoncriteriaportal.org/files/epfiles/Document%20SAFER%20Blue%202%20Security%20Target%20V1.10_???.pdf, code: nok
Cert dgst: ace7ea6d7f58dbb1 failed to download report from https://www.commoncriteriaportal.org/files/epfiles/0308a_pdf.pdf, code: nok
Cert dgst: 305af0eec85bb856 failed to download report from https://www.commoncriteriaportal.org/files/epfiles/[KEC

In [10]:
# Convert all
dset.convert_all_pdfs()

Converting reports to txt:   2%|██                                                                                         | 118/5115 [00:06<04:56, 16.83it/s]Error when converting pdf->txt: poppler error creating document
Detected garbage during conversion of cc_09_10_2022/certs/reports/pdf/bfc86f14e2484335.pdf
Error during OCR of cc_09_10_2022/certs/reports/pdf/bfc86f14e2484335.pdf, using garbage: pdftoppm failed: 1
Cert dgst: bfc86f14e2484335 failed to convert report pdf->txt
Converting reports to txt:   4%|███▍                                                                                       | 194/5115 [00:10<04:03, 20.19it/s]Detected garbage during conversion of cc_09_10_2022/certs/reports/pdf/d8c205b4924f91b3.pdf
Detected garbage during conversion of cc_09_10_2022/certs/reports/pdf/f8a79eb20ade29af.pdf
Detected garbage during conversion of cc_09_10_2022/certs/reports/pdf/3477723044183b31.pdf
Detected garbage during conversion of cc_09_10_2022/certs/reports/pdf/82c24f729c2e0092

Converting reports to txt:  17%|███████████████                                                                            | 849/5115 [22:42<09:07,  7.79it/s]Error when converting pdf->txt: poppler error creating document
Detected garbage during conversion of cc_09_10_2022/certs/reports/pdf/4de4a9f436958574.pdf
Error when converting pdf->txt: poppler error creating document
Detected garbage during conversion of cc_09_10_2022/certs/reports/pdf/98668cafb8dda26f.pdf
Error during OCR of cc_09_10_2022/certs/reports/pdf/98668cafb8dda26f.pdf, using garbage: pdftoppm failed: 1
Cert dgst: 98668cafb8dda26f failed to convert report pdf->txt
Error when converting pdf->txt: poppler error creating document
Detected garbage during conversion of cc_09_10_2022/certs/reports/pdf/02958808b57334f0.pdf
Error during OCR of cc_09_10_2022/certs/reports/pdf/02958808b57334f0.pdf, using garbage: pdftoppm failed: 1
Cert dgst: 02958808b57334f0 failed to convert report pdf->txt
Error during OCR of cc_09_10_2022/cer

Cert dgst: f9c8da9deff77ab5 failed to convert report pdf->txt
Converting reports to txt:  87%|█████████████████████████████████████████████████████████████████████████████▉            | 4430/5115 [29:05<00:53, 12.70it/s]Detected garbage during conversion of cc_09_10_2022/certs/reports/pdf/40640508ac3a37f2.pdf
Error when converting pdf->txt: poppler error creating document
Converting reports to txt:  87%|██████████████████████████████████████████████████████████████████████████████▏           | 4444/5115 [29:07<00:57, 11.59it/s]Detected garbage during conversion of cc_09_10_2022/certs/reports/pdf/7147c2f70d983d57.pdf
Error during OCR of cc_09_10_2022/certs/reports/pdf/7147c2f70d983d57.pdf, using garbage: pdftoppm failed: 1
Cert dgst: 7147c2f70d983d57 failed to convert report pdf->txt
Converting targets to txt:   2%|██                                                                                         | 117/5038 [00:29<20:18,  4.04it/s]Detected garbage during conversion of cc_09_10_2

Error when converting pdf->txt: poppler error creating document
Error when converting pdf->txt: poppler error creating document
Error when converting pdf->txt: poppler error creating document
Error during OCR of cc_09_10_2022/certs/targets/pdf/7147c2f70d983d57.pdf, using garbage: pdftoppm failed: 1
Error when converting pdf->txt: poppler error creating document
Detected garbage during conversion of cc_09_10_2022/certs/targets/pdf/a2b962c7f1d2bc56.pdf
Detected garbage during conversion of cc_09_10_2022/certs/targets/pdf/422dc5758723c7d1.pdf
Detected garbage during conversion of cc_09_10_2022/certs/targets/pdf/bfc86f14e2484335.pdf
Cert dgst: 7147c2f70d983d57 failed to convert security target pdf->txt
Detected garbage during conversion of cc_09_10_2022/certs/targets/pdf/4c9468f20fdb04f7.pdf
Error during OCR of cc_09_10_2022/certs/targets/pdf/a2b962c7f1d2bc56.pdf, using garbage: pdftoppm failed: 1
Cert dgst: a2b962c7f1d2bc56 failed to convert security target pdf->txt
Error during OCR of cc

In [11]:
# Analyze all
dset.analyze_certificates()

Extracting report metadata:   0%|                                                                                                    | 0/5107 [00:00<?, ?it/s]Failed to read metadata of certs/reports/pdf/1e91595cd5e7fece.pdf, error: Unexpected escaped string: b'\x83'
Failed to read metadata of certs/reports/pdf/451e0806dfcfd29d.pdf, error: PDF metadata unavailable
Extracting report metadata:  33%|████████████████████████████▉                                                           | 1677/5107 [00:16<00:23, 147.61it/s]Failed to read metadata of certs/reports/pdf/a62c55533f4e1636.pdf, error: PDF metadata unavailable
Failed to read metadata of certs/reports/pdf/d3568613c552f9e8.pdf, error: EOF marker not found
Extracting report metadata:  44%|███████████████████████████████████████                                                 | 2269/5107 [00:19<00:16, 175.34it/s]Failed to read metadata of certs/reports/pdf/1c5deb472a197eb6.pdf, error: PDF metadata unavailable
Failed to read metadata o

Extracting target metadata:  32%|████████████████████████████▏                                                            | 1595/5033 [00:48<01:23, 41.13it/s]Failed to read metadata of certs/targets/pdf/bbcfe0aee7ea78ea.pdf, error: PDF metadata unavailable
Failed to read metadata of certs/targets/pdf/f0bd6a29eee94a2c.pdf, error: PDF metadata unavailable
Failed to read metadata of certs/targets/pdf/463ecd64b7506048.pdf, error: PDF metadata unavailable
Predicting CPE matches with the classifier:  33%|███████████████████████▉                                                 | 1686/5129 [00:53<01:43, 33.41it/s]

geee
geee
geee
geee
geee


parsing cpe matching (by NIST) dictionary: 100%|███████████████████████████████████████████████████████████████████| 398786/398786 [00:26<00:00, 15135.38it/s]
Building-up lookup dictionaries for fast CVE matching: 100%|███████████████████████████████████████████████████████| 197102/197102 [00:18<00:00, 10511.33it/s]
Computing related CVES: 100%|████████████████████████████████████████████████████████████████████████████████████████████| 650/650 [00:00<00:00, 70385.13it/s]


In [12]:
# And do maintenance updates as well
dset.process_maintenance_updates()

Converting reports to txt:  95%|███████████████████████████████████████████████████████████████████████████████████████▍    | 436/459 [00:10<00:00, 41.44it/s]ERROR:sec_certs.utils.pdf:Error during OCR of cc_09_10_2022/certs/maintenances/reports/pdf/cert_8ba5d4d02ea73d3d_update_d879046e9be61396.pdf, using garbage: pdftoppm failed: 1
ERROR:sec_certs.sample.certificate:Cert dgst: cert_8ba5d4d02ea73d3d_update_d879046e9be61396 failed to convert report pdf->txt
Converting reports to txt:   0%|                                                                                                        | 0/1 [00:00<?, ?it/s]ERROR:sec_certs.utils.pdf:Error when converting pdf->txt: poppler error creating document
ERROR:sec_certs.utils.pdf:Error during OCR of cc_09_10_2022/certs/maintenances/reports/pdf/cert_8ba5d4d02ea73d3d_update_d879046e9be61396.pdf, using garbage: pdftoppm failed: 1
ERROR:sec_certs.sample.certificate:Cert dgst: cert_8ba5d4d02ea73d3d_update_d879046e9be61396 failed to convert report

In [13]:
# Finally, dump it all
dset.to_json()